# Depedency Initialization
----

In [1]:
!pip install -q streamlit

In [2]:
!pip install -qU langchain langchain-community langchain-openai langchain-text-splitters langchain-classic streamlit faiss-cpu unstructured[local-inference]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.9 MB/s eta 0:00:00


In [3]:
!pip uninstall -y transformers huggingface-hub sentence-transformers langchain-huggingface

!pip install -q "transformers==4.46.2" "sentence-transformers==2.7.0" langchain-huggingface


Found existing installation: transformers 5.2.0
Uninstalling transformers-5.2.0:
  Successfully uninstalled transformers-5.2.0
Found existing installation: huggingface_hub 1.5.0
Uninstalling huggingface_hub-1.5.0:
  Successfully uninstalled huggingface_hub-1.5.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.8 MB/s eta 0:00:00


In [4]:
import os
import streamlit as st
import pickle
import time

# ✅ 2026 LANGCHAIN IMPORTS
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_classic.chains import RetrievalQAWithSourcesChain  # langchain-classic
from langchain_classic.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import UnstructuredURLLoader
print("✅ ALL IMPORTS WORKING - News Agent Ready!")


✅ ALL IMPORTS WORKING - News Agent Ready!


In [5]:
import transformers
print("Transformers version:", transformers.__version__)
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
print("Import OK")


Transformers version: 4.46.2


Import OK


# LLM Pipeline Creation
---

In [23]:
# 2. ✅ LOCAL LLM - No HF API issues!
# import langchain
# langchain.debug = True
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Small, fast model for Colab CPU
model_id = "google/flan-t5-small"  # Or "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Local pipeline
hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7,
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Data Loader
---

In [24]:
# llm = ChatOpenAI(temperature=0.9, max_tokens=500)


loaders = UnstructuredURLLoader(urls= [
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])

data = loaders.load()
len(data)

2

# Text Splitting
----

In [25]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(data)
len(docs)

18

# OpenAI Embedding
----

In [ ]:
#OpenAI token embedding
embeddings = OpenAIEmbeddings() #skip this cell if you are using the free huugingFace embedding
vectorindex_openai = FAISS.from_documents(docs,embeddings)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# Free Hugging Face Embedding
---

In [26]:
#Free Hugging Face Embedding
# FREE OpenAI-compatible embeddings

!pip install -q sentence-transformers

from langchain_community.embeddings import HuggingFaceEmbeddings

# Replace your OpenAI embeddings with FREE:
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"  # Fast, good quality
)

vectorindex_openai = FAISS.from_documents(docs, embeddings)
print("✅ FREE embeddings working!")

✅ FREE embeddings working!


In [27]:
file_path = "vector_index.pkl"
with open(file_path,"wb") as f:
    pickle.dump(vectorindex_openai,f)


In [28]:
if os.path.exists(file_path):
    with open(file_path,"rb") as f:
        vectorIndex = pickle.load(f)

# Chain creation with RAG
---

In [29]:
#OpenAI Retrival Chain
# chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex_openai.as_retriever())

# chain
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs, embeddings)

#HuggingFace / Ollama retrival chain
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,  # FREE LLM
    chain_type="stuff",
    retriever=vectorstore.as_retriever(
    search_kwargs={"k": 2}),
    return_source_documents=True
)
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, partial_variables={}, template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").\nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or del

# Search Query Test
---

In [30]:
import langchain

ques = "what is the price of of Tiago iCNGZ?"

langchain.debug = True

# chain({"question":ques}, return_only_outputs=True) #For OpenAI from_llm

# If your chain is a RetrievalQAWithSourcesChain or similar:
result = chain.invoke({"question": ques}, return_only_outputs=True)
print(result["answer"])

Token indices sequence length is longer than the specified maximum sequence length for this model (2047 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Tata Motors launched Punch iCNG, price starts at 7.1 lakh.
